In [2]:
import sys
sys.path.append('/scratch/pradap/python-work/dmagellan/')

In [3]:
from dmagellan.blocker.overlap.overlapblocker_dask import *

In [4]:
import pandas as pd
from dask import dataframe as dd

In [5]:
A = dd.read_csv('./datasets/sample_msd_100k.csv', blocksize=5000000)
B = dd.read_csv('./datasets/sample_msd_100k.csv', blocksize=5000000)

In [6]:
ob = OverlapBlocker()
C = ob.block_tables(A, B, 'id', 'id', 'title', 'title', rem_stop_words=True, overlap_size=3)

In [7]:
x = [0, 0, 0]
meta = pd.DataFrame([[0, 0, 0]], columns=['_id', 'l_id', 'r_id'])

In [8]:
D = dd.from_delayed(C, meta=meta)

In [10]:
list(A.columns) + ['_id']

['id',
 'title',
 'release',
 'artist_name',
 'duration',
 'artist_familiarity',
 'artist_hotttnesss',
 'year',
 '_id']

In [11]:
A.dtypes

id                      int64
title                  object
release                object
artist_name            object
duration              float64
artist_familiarity    float64
artist_hotttnesss     float64
year                    int64
dtype: object

In [13]:
x = [0, '', '', '', 0.0, 0.0, 0.0, 0, 0]
c = list(A.columns) + ['_id']
meta = pd.DataFrame([x], columns = c)

In [24]:
lens = A.map_partitions(len)

In [32]:
def get_length(df):
    return len(df)
def get_lengths(df):
    res = []
    for i in range(df.npartitions):
        l = (delayed)(get_length)(df.get_partition(i))
        res.append(l)
    return res

In [33]:
lens = get_lengths(A)

In [42]:
def compute_ranges(lengths):
    res = [0]*(len(lengths)+1)
    for i in range(len(lengths)):
        res[i+1] = res[i] + lengths[i]
#         res.append(val)
    return res

In [43]:

compute_ranges(l)

[0, 100, 300, 700, 900]

In [45]:
kk = A.get_partition(0)

In [46]:
pp = kk.compute()

In [48]:
pp.insert(0, '_id', range(len(pp)))

In [49]:
pp.head()

,_id,id,title,release,artist_name,duration,artist_familiarity,artist_hotttnesss,year
0,0,622483,Battery,Chaos Theory,Amon Tobin,266.16118,0.757409,0.497130,2005
1,1,459437,44 Blues,"Songs of Memphis Slim and ""Wee Willie"" Dixon",Memphis Slim and Willie Dixon,156.08118,0.398566,0.292463,0
2,2,638163,Religion II,Public Image,Public Image Ltd,352.44363,0.684995,0.415517,1978
3,3,509809,Silent night,Kimera chante noël,Kiméra,118.75220,0.504995,0.319393,0
4,4,476164,Lotus Blossom,The Fix Is In,James Chance & Terminal City,392.75057,0.464823,0.364014,0


In [89]:
def add_id(df, start, end):
    df.insert(0, '_id', range(start, end))
    return df
    

In [85]:
x = pd.read_csv('datasets/person_table_A.csv')

In [86]:
x.insert(0, '_id', range(len(x)))

In [87]:
x

,_id,ID,name,birth_year,hourly_wage,address,zipcode
0,0,0,Kevin Smith,1989,30.0,"607 From St, San Francisco",94107
1,1,1,Michael Franklin,1988,27.5,"1652 Stockton St, San Francisco",94122
2,2,2,William Bridge,1986,32.0,"3131 Webster St, San Francisco",94107
3,3,3,Binto George,1987,32.5,"423 Powell St, San Francisco",94122
4,4,4,Alphonse Kemper,1984,35.0,"1702 Post Street, San Francisco",94122


In [51]:
# help(A.map_partitions)

In [60]:
def apply_ranges(df, ranges):
    results = []
    for i in range(df.npartitions):
        res = delayed(add_id)(df.get_partition(i), ranges[i], ranges[i+1])
        results.append(res)
    return results
    
        

In [90]:
lens = get_lengths(A)
ranges = delayed(compute_ranges)(lens)
result = apply_ranges(A, ranges)


In [91]:
xx = result[0].compute()

In [92]:
xx

,_id,id,title,release,artist_name,duration,artist_familiarity,artist_hotttnesss,year
0,0,622483,Battery,Chaos Theory,Amon Tobin,266.16118,0.757409,0.497130,2005
1,1,459437,44 Blues,"Songs of Memphis Slim and ""Wee Willie"" Dixon",Memphis Slim and Willie Dixon,156.08118,0.398566,0.292463,0
2,2,638163,Religion II,Public Image,Public Image Ltd,352.44363,0.684995,0.415517,1978
3,3,509809,Silent night,Kimera chante noël,Kiméra,118.75220,0.504995,0.319393,0
4,4,476164,Lotus Blossom,The Fix Is In,James Chance & Terminal City,392.75057,0.464823,0.364014,0
5,5,111794,Real enemy(The Business),The Worldwide Tribute to the real Oi Vol.2,Funeral Dress,161.09669,0.554503,0.418868,0
6,6,252878,Upside Downside,Upside Downside,Mike Stern,340.79302,0.636510,0.382038,0
7,7,123770,Forked Tongue,Uphill Battle,Uphill Battle,197.72036,0.466262,0.369610,2002
8,8,831389,Right On Time,Ay! Mi Vida,Jerry Rivera,237.19138,0.650177,0.404963,2005
9,9,365354,Theme For An Ultimate And Inevitable Victory,No Room For Humans,Servotron,102.34730,0.480775,0.370381,1996


In [84]:
type(result[0].compute())

NoneType

In [74]:
ranges.compute()

[0, 48023, 96022, 100000]

In [75]:
xx = result[0].compute()

In [77]:
A.head()

,id,title,release,artist_name,duration,artist_familiarity,artist_hotttnesss,year
0,622483,Battery,Chaos Theory,Amon Tobin,266.16118,0.757409,0.497130,2005
1,459437,44 Blues,"Songs of Memphis Slim and ""Wee Willie"" Dixon",Memphis Slim and Willie Dixon,156.08118,0.398566,0.292463,0
2,638163,Religion II,Public Image,Public Image Ltd,352.44363,0.684995,0.415517,1978
3,509809,Silent night,Kimera chante noël,Kiméra,118.75220,0.504995,0.319393,0
4,476164,Lotus Blossom,The Fix Is In,James Chance & Terminal City,392.75057,0.464823,0.364014,0


In [72]:
get_length(A.get_partition(0))

48023

In [93]:
x = [0, 0,  '', '', '', 0.0, 0.0, 0.0, 0]
c = ['_id'] + list(A.columns) 
meta = pd.DataFrame([x], columns = c)
final = dd.from_delayed(result, meta=meta)

In [94]:
final.head()

,_id,id,title,release,artist_name,duration,artist_familiarity,artist_hotttnesss,year
0,0,622483,Battery,Chaos Theory,Amon Tobin,266.16118,0.757409,0.497130,2005
1,1,459437,44 Blues,"Songs of Memphis Slim and ""Wee Willie"" Dixon",Memphis Slim and Willie Dixon,156.08118,0.398566,0.292463,0
2,2,638163,Religion II,Public Image,Public Image Ltd,352.44363,0.684995,0.415517,1978
3,3,509809,Silent night,Kimera chante noël,Kiméra,118.75220,0.504995,0.319393,0
4,4,476164,Lotus Blossom,The Fix Is In,James Chance & Terminal City,392.75057,0.464823,0.364014,0


In [95]:
final.tail()

,_id,id,title,release,artist_name,duration,artist_familiarity,artist_hotttnesss,year
3973,99995,856394,Shoot shoot,Bail set at $6M,Batmobile,204.59057,0.632946,0.430789,1988
3974,99996,707067,Prends ma main,Crazy Horse - L'essentiel,Crazy Horse,186.30485,0.540280,0.342192,0
3975,99997,325602,Shoo-Ra,Swamp Blues,Dr. John,187.34975,0.736282,0.496459,0
3976,99998,122347,Stereo Flo (Club Mix Radio Edit),Stereo Flo,Dada_ Obernik & Harris,219.34975,0.329658,0.438285,0
3977,99999,469239,Angels We Have Heard On High (Instrumental) (I...,Noel,Joan Baez,82.52036,0.732261,0.455289,0


In [98]:
x = A.repartition(4)

ValueError: New division must be list or tuple

In [111]:
x = A.repartition(npartitions=8)

In [112]:
x.map_partitions(len).compute()

0    24011
1    24012
2    23999
3    24000
4      994
5      995
6      994
7      995
dtype: int64